In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1595711/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_42_3,0.998699,0.817918,0.763240,0.989662,0.005631,1.217581,0.846878,0.017145,0.147634,0.075041,1.083236,0.076222,140.358906,281.694078,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_45_1,0.998637,0.812909,0.723816,0.998223,0.005900,1.251080,0.987896,0.006833,0.174674,0.076814,1.087215,0.078023,140.265496,281.600669,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_49_4,0.998635,0.798415,0.737350,0.995624,0.005908,1.348001,0.939485,0.018477,0.104092,0.076864,1.087331,0.078075,140.262848,281.598021,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_48_9,0.998630,0.805589,0.736871,0.993897,0.005933,1.300030,0.941197,0.021339,0.133361,0.077024,1.087693,0.078237,140.254566,281.589739,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_49_5,0.998581,0.797275,0.736632,0.995158,0.006142,1.355622,0.942053,0.020444,0.105420,0.078374,1.090794,0.079608,140.185075,281.520248,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,model_1_3_7,0.888237,0.619578,0.145976,0.618746,0.483905,2.543886,3.054794,2.050877,1.798209,0.695633,8.152814,0.706588,131.451735,272.786908,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
417,model_1_10_7,0.887755,-1.577349,-0.554570,-2.033647,0.485991,17.234748,5.560608,2.264153,1.348123,0.697131,8.183657,0.708110,131.443129,272.778302,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
418,model_1_17_2,0.886018,0.687724,0.675077,0.689093,0.493513,2.088191,1.162230,1.041620,1.656844,0.702505,8.294846,0.713569,131.412410,272.747583,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
419,model_1_17_3,0.885383,0.641065,0.302441,0.514043,0.496262,2.400200,2.495129,1.628085,1.574692,0.704458,8.335468,0.715553,131.401304,272.736477,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
